In [1]:
import os
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers, callbacks
import tensorflow.keras.backend as K

%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

AssertionError: Not enough GPU hardware devices available

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [17]:
image_width = 224
image_height = 224
num_classes = 101
histories = []

In [18]:
images_dir = "images/"
train_data_dir = images_dir 
val_data_dir = images_dir
test_data_dir = images_dir 

In [19]:
checkpoint_dir = "checkpoints/"
checkpoint_name = checkpoint_dir + "multisnacks-{val_loss:.4f}-{val_accuracy:.4f}.hdf5"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [20]:
def create_callbacks():
    return [
        callbacks.EarlyStopping(monitor="val_accuracy", patience=10, verbose=1),

        callbacks.ModelCheckpoint(checkpoint_name, monitor="val_accuracy", 
                                  verbose=1, save_best_only=True),
    ]

my_callbacks = create_callbacks()

In [21]:
my_callbacks = create_callbacks()

In [22]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

base_model = MobileNetV2(input_shape=(image_height, image_width, 3), 
                       include_top=False, weights="imagenet", 
                       pooling=None)

In [23]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

train_datagen = ImageDataGenerator(
                    rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    channel_shift_range=0.2,
                    horizontal_flip=True,
                    fill_mode="nearest",
                    preprocessing_function=preprocess_input)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [24]:
batch_size = 64

train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(image_width, image_height),
                    batch_size=batch_size,
                    class_mode="categorical",
                    shuffle=True)

val_generator = val_datagen.flow_from_directory(
                    val_data_dir,
                    target_size=(image_width, image_height),
                    batch_size=batch_size,
                    class_mode="categorical",
                    shuffle=False)

test_generator = test_datagen.flow_from_directory(
                    test_data_dir,
                    target_size=(image_width, image_height),
                    batch_size=batch_size,
                    class_mode="categorical",
                    shuffle=False)

Found 101000 images belonging to 101 classes.
Found 101000 images belonging to 101 classes.
Found 101000 images belonging to 101 classes.


In [25]:
from tensorflow.keras import regularizers

top_model = Sequential()
top_model.add(base_model)
top_model.add(GlobalAveragePooling2D())
top_model.add(Dropout(0.7))
top_model.add(Dense(num_classes, kernel_regularizer=regularizers.l2(0.01)))
top_model.add(Activation("softmax"))

In [26]:
for layer in base_model.layers:
    layer.trainable = False

top_model.compile(loss="categorical_crossentropy",
                  optimizer=optimizers.Adam(lr=1e-4),
                  metrics=["accuracy"])    

In [ ]:
histories.append(top_model.fit(train_generator, 
                              steps_per_epoch=len(train_generator),
                              epochs=60,
                              callbacks=my_callbacks,
                              validation_data=val_generator,
                              validation_steps=len(val_generator),
                              workers=8))

Epoch 1/60
  56/1579 [>.............................] - ETA: 11:46 - loss: 7.7259 - accuracy: 0.0117

## Evaluate on the best model

In [35]:
from keras.models import load_model
model_saved_name = ""

best_model = load_model(checkpoint_dir + "multisnacks-0.1689-1.0000.hdf5")

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [29]:
best_model.evaluate_generator(test_generator, steps=len(test_generator))

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.16889998316764832, 1.0]

In [30]:
test_generator.reset()
probabilities = best_model.predict_generator(test_generator, steps=len(test_generator))
predicted_labels = np.argmax(probabilities, axis=-1)

Instructions for updating:
Please use Model.predict, which supports generators.


In [31]:
target_labels = test_generator.classes

In [32]:
from sklearn import metrics
conf = metrics.confusion_matrix(target_labels, predicted_labels)


In [33]:
import seaborn as sns

def plot_confusion_matrix(conf, labels, figsize=(8, 8)):
    fig = plt.figure(figsize=figsize)
    heatmap = sns.heatmap(conf, annot=True, fmt="d")
    heatmap.xaxis.set_ticklabels(labels, rotation=45, ha="right", fontsize=12)
    heatmap.yaxis.set_ticklabels(labels, rotation=0, ha="right", fontsize=12)
    plt.xlabel("Predicted label", fontsize=12)
    plt.ylabel("True label", fontsize=12)
    plt.show()

In [ ]:
labels = [""] * num_classes
for k, v in test_generator.class_indices.items():
    labels[v] = k

In [ ]:
plot_confusion_matrix(conf, labels, figsize=(14, 14))

In [ ]:
print(metrics.classification_report(target_labels, predicted_labels, target_names=labels))

In [ ]:
# Find for which images the predicted class is wrong
wrong_images = np.where(predicted_labels != target_labels)[0]

# For every prediction, find the largest probability value;
# this is the probability of the winning class for this image
probs_max = np.max(probabilities, axis=-1)

# Sort the probabilities from the wrong images from low to high
idx = np.argsort(probs_max[wrong_images])

# Reverse the order (high to low), and keep the 5 highest ones
idx = idx[::-1][:5]

# Get the indices of the images with the worst predictions
worst_predictions = wrong_images[idx]

index2class = {v:k for k,v in test_generator.class_indices.items()}

for i in worst_predictions:
    print("%s was predicted as '%s' %.4f\n" % (
        test_generator.filenames[i],
        index2class[predicted_labels[i]],
        probs_max[i]
    ))

In [ ]:
from keras.preprocessing import image
img = image.load_img(test_data_dir + test_generator.filenames[worst_predictions[0]])
plt.imshow(img)

## Export to Core ML

Restart kernel with mlvenv with "source mlvenv/bin/activate" for coremltools part

In [17]:
import coremltools as ct
import os
from tensorflow.keras.models import load_model

In [18]:
label_dir = "ava/"
labels = []
checkpoint_dir = "checkpoints/"
best_model = tensorflow.keras.load_model(checkpoint_dir + "multisnacks-1.6708-0.4054.hdf5")

NameError: name 'tensorflow' is not defined

In [3]:
for i in os.listdir(label_dir):
    if i != '.DS_Store' and i != '._.DS_Store':
        labels.append(i)

In [4]:
coreml_model = ct.convert(
    best_model,
    input_names=["image"],
    image_input_names="image",
    output_names="labelProbability",
    predicted_feature_name="label",
    red_bias=-1,
    green_bias=-1,
    blue_bias=-1,
    image_scale=2/255.0,
    class_labels=labels)

Translating MIL ==> MLModel Ops: 100%|██████████| 701/701 [00:00<00:00, 3765.88 ops/s]


In [5]:
coreml_model.author = "Joshua Ball"
coreml_model.short_description = "Image classifier"

coreml_model.input_description["image"] = "Input image"
coreml_model.output_description["labelProbability"]= "Prediction probabilities"
coreml_model.output_description["label"]= "Class label of top prediction"

AttributeError: No feature with name image.

In [6]:
coreml_model.save("MultiSnacks.mlmodel")

In [7]:
m = ct.models.MLModel("MultiSnacks.mlmodel")

In [8]:
import numpy as np
import PIL.Image


In [9]:
Height = 224  # use the correct input image height
Width = 224  # use the correct input image width

In [10]:
def load_image(path, resize_to=None):
    # resize_to: (Width, Height)
    img = PIL.Image.open(path)
    if resize_to is not None:
        img = img.resize(resize_to, PIL.Image.ANTIALIAS)
    img_np = np.array(img).astype(np.float32)
    return img_np, img


In [13]:
_, img = load_image('ava/Ripe/focused_184465770-stock-photo-whole-ripe-avocado.jpg', resize_to=(Width, Height))
out_dict = m.predict({'image': img})


Exception: Model prediction is only supported on macOS version 10.13 or later.